# Azure ML - Jupyter Notebooks

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [1]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_name = "azure-ml-workspace-Python-SDK"

ml_client = MLClient(DefaultAzureCredential(), AZURE_SUSCRIPION_ID, AZURE_ML_RESOURCE_GRPU_ID, workspace_name)

## Jupyter Notebooks

Esta parte es la haremos con la interfaz gráfica, por lo que hazlo con el `Workspace` que quieras

En Studio seleccionamos `Notebooks` en la parte izquierda de la interfaz. Vemos que tenemos una zona con las carpetas y otra en la que podemos darle al botón de `+ Files` para crear nuevos archivos, de modo que le damos al botón, seleccionamos `Create new file`, ponemos un nombre, en mo caso pondré `baseline.ipynb`, en `File type` seleccionamos `Notebook` y le damos a `Create`

Vemos que se nos ha creado un Jupyter Notebook

### Seleccionar un `Compute Instance`

Lo primero que tenemos que hacer es seleccionar un `Compute Instance` para ello pinchar en la zona de `Compute` de la parte superior del notebook. Nos aparecerán los `Compute Instance`s que hayamos creado, seleccionamos uno y le damos al botón de `Start`, tardará un poco en arrancar

### Kernel

Una vez hemos levantado una `Compute Instance` tenemos que elegir un kernel, igual que cuando ejecutamos un Jupyter Notebook en local. Para ellos pinchamos en la zona de `Kernel` en la parte superior del notebook y seleccionamos el kernel que queramos. En mo caso voy a seleccionar `Python 3.10 - SDK v2`

Este kernel es un entorno de conda, por lo que podemos crearnos nuevos si queremos. En la zona donde se ven las carpetas hay un botón con formade una terminal, si dejamos el botón encima aparece el texto `Open terminal`. Si le damos se nos abrirá la terminal de esa `Compute Instance`, por lo que podríamos crear nuevos entornos de conda si queremos. Luego podremos seleccionar esos entornos como kernel

En mi caso voy a usar el kernel `Python 3.10 - SDK v2`, abro la terminal e instalo las siguientes librerías

```bash
conda activate azureml_py310_sdkv2
conda install -y pytorch torchvision pytorch-cuda=12.4 -c pytorch -c nvidia
pip install -U transformers accelerate datasets
```

Desafortunadamente, aunque estemos en `Azure ML`, hayamos creado un `Compute Instance` con GPU y esa máquina esté corriendo sobre Ubuntu (si, hasta Microsoft sabe que Linux es mejor), no tiene instalados los drivers de Nvidia, por lo que si haces 

```python
import torch
torch.cuda.is_available()
```

Siempre va a dar `False` así que todo lo que hagas va a ser sobre CPU

### Dónde editar el Jupyter Notebook

También, cuando levantamos el `Compute Instance` se habilita una zona en la parte superior del notebook que pone `Edit in VS Code`. Si le damos se abrirá un menú en el que podemos seleccionar si editarlo en vscode en la web o en local. Haz lo que prefieras

### Datos

Ya solo nos faltan los datos antes de ponernos a entrenar, para ello, en la zona de las carpetas hay un botón con el símbolo `+`, si le damos podemos subir archivos. Vamos a subir la carpeta `en` que descargamos de HuggingFace, marcamos la casilla que dice si creemos en los autores y le damos a `Upload`. Ya tenemos los datos subidos

### Entrenamiento

A continuación pondré el notebook que he usado para entrenar el modelo. Cómo he dicho corre sobre una CPU, por lo que voy a crear un subconjunto del dataset muy pequeño y entrenar solo por 1 época. El porpósito del post no es explicar cómo entrenar el modelo, este mismo modelo ya lo explico en el post [fine-tuning-sml](https://www.maximofn.com/fine-tuning-sml#Fine-tuning-para-clasificaci%C3%B3n-de-texto-con-Pytorch)

#### Dataset

Vemos dónde tenemos los datos

In [ ]:
!ls

baseline.ipynb	baseline.ipynb.amltmp  en


In [ ]:
!ls en

en_test.jsonl  en_validation.jsonl  train.jsonl


Creamos los datasets

In [ ]:
from datasets import load_dataset

dataset_train = load_dataset("json", data_files="en/train.jsonl")
dataset_validation = load_dataset("json", data_files="en/en_validation.jsonl")
dataset_test = load_dataset("json", data_files="en/en_test.jsonl")

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vamos a verlos

In [ ]:
dataset_train

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
})

In [ ]:
dataset_validation

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})

In [ ]:
dataset_test

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})

Vemos que tiene un conjunto de entrenamiento con 200.000 muestras, uno de validación con 5.000 muestras y uno de test de 5.000 muestras

Vamos a ver un ejemplo del conjunto de entrenamiento

In [ ]:
from random import randint
idx = randint(0, len(dataset_train['train']) - 1)
dataset_train['train'][idx]

{'id': 'en_0531097',
 'text': 'One Star\n\nDid not receive a black clip as the product and packaging advertises.',
 'label': 0,
 'label_text': '0'}

Vemos que tiene la review en el campo `text` y la puntuación que le ha dado el usuario en el campo `label`

Como vamos a hacer un modelo de clasificación de textos, necesitamos saber cuantas clases vamos a tener

In [ ]:
num_classes = len(dataset_train['train'].unique('label'))
num_classes

5

Vamos a tener 5 clases, ahora vamos a ver el valor mínimo de estas clases para saber si la puntuación comienza en 0 o en 1. Para ello usamos el método `unique`

In [ ]:
dataset_train.unique('label')

{'train': [0, 1, 2, 3, 4]}

El mínimo valor va a ser 0

#### Tokenizador

Como en el dataset tenemos las reviews en texto, necesitamos tokenizarlas para poder meter los tokens al modelo

In [ ]:
from transformers import AutoTokenizer
      
checkpoint = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

Ahora creamos una función para tokenizar el texto. Lo vamos a hacer de manera que todas las sentencias tenan la mismas longitud, de manera que el tokenizador truncará cuando sea necesario y añadirá tokens de padding cuando sea necesario. Además le indicamos que devuelva tensores de pytorch

Hacemos que la longitud de cada sentencia sea de 768 tokens porque estamos usando el modelo pequeño de GPT2, que como vimos en el post de [GPT2](https://www.maximofn.com/gpt2/#Arquitectura) tiene una dimensión de embedding de 768 tokens

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=768, return_tensors="pt")

Vamos a probar a tokenizar un texto

In [ ]:
tokens = tokenize_function(dataset_train['train'][idx])
tokens['input_ids'].shape, tokens['attention_mask'].shape

(torch.Size([1, 768]), torch.Size([1, 768]))

Ahora que hemos comprobado que la función tokeniza bien, aplicamos esta función al dataset, pero además la aplicamos por batches para que se ejecute más rápido

Además aprovechamos y eliminamos las columnas que no vamos a necesitar

In [ ]:
dataset_train = dataset_train.map(tokenize_function, batched=True, remove_columns=['id', 'label_text'])
dataset_validation = dataset_validation.map(tokenize_function, batched=True, remove_columns=['id', 'label_text'])
dataset_test = dataset_test.map(tokenize_function, batched=True, remove_columns=['id', 'label_text'])

In [ ]:
dataset_train

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 200000
    })
})

In [ ]:
dataset_validation

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [ ]:
dataset_test

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

Vemos que tenemos los campos `labels`, `input_ids` y `attention_mask`, que es lo que nos interesa para entrenar

Creamos un subset

In [ ]:
percentage = 0.0001
subset_train = dataset_train['train'].select(range(int(len(dataset_train['train']) * percentage)))
percentage = 0.001
subset_validation = dataset_validation['train'].select(range(int(len(dataset_validation['train']) * percentage)))
subset_test = dataset_test['train'].select(range(int(len(dataset_test['train']) * percentage)))
print(f"len subset_train: {len(subset_train)}, len subset_validation: {len(subset_validation)}, len subset_test: {len(subset_test)}")

len subset_train: 20, len subset_validation: 5, len subset_test: 5


#### Modelo

Instanciamos un modelo para clasificación de secuencias y le indicamos el número de clases que tenemos. De momento para entrenar rápido usamos GPT2

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_classes)
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Device

Creamos el dispositivo donde se va a ejecutar todo

In [ ]:
import torch
      
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

De paso pasamos el modelo al dispositivo y de paso lo pasamos a FP16 para que ocupe menos memoria

In [ ]:
model.half().to(device)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=5, bias=False)
)

#### Pytorch Dataset

Creamos un dataset de pytorch

In [ ]:
from torch.utils.data import Dataset
      
class ReviewsDataset(Dataset):
    def __init__(self, huggingface_dataset):
        self.dataset = huggingface_dataset

    def __getitem__(self, idx):
        label = self.dataset[idx]['label']
        input_ids = torch.tensor(self.dataset[idx]['input_ids'])
        attention_mask = torch.tensor(self.dataset[idx]['attention_mask'])
        return input_ids, attention_mask, label

    def __len__(self):
        return len(self.dataset)

Instanciamos los datasets

In [ ]:
train_dataset = ReviewsDataset(subset_train)
validatation_dataset = ReviewsDataset(subset_validation)
test_dataset = ReviewsDataset(subset_test)

Vamos a ver una muestra

In [ ]:
input_ids, at_mask, label = train_dataset[0]
input_ids.shape, at_mask.shape, label

(torch.Size([768]), torch.Size([768]), 0)

#### Pytorch Dataloader

Creamos ahora un dataloader de pytorch

In [ ]:
from torch.utils.data import DataLoader
      
BS = 2

train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
validation_loader = DataLoader(validatation_dataset, batch_size=BS)
test_loader = DataLoader(test_dataset, batch_size=BS)

Vamos a ver una muestra

In [ ]:
input_ids, at_mask, labels = next(iter(train_loader))
input_ids.shape, at_mask.shape, labels

(torch.Size([2, 768]), torch.Size([2, 768]), tensor([0, 0]))

Para ver que está todo bien pasamos la muestra al modelo para ver qué sale todo bien. Primero pasamos los tokens al dispositivo

In [ ]:
input_ids = input_ids.to(device)
at_mask = at_mask.to(device)
labels = labels.to(device)

Ahora se los pasamos al modelo

In [ ]:
output = model(input_ids=input_ids, attention_mask=at_mask, labels=labels)
output.keys()

odict_keys(['loss', 'logits', 'past_key_values'])

Como vemos nos da la loss y los logits

In [ ]:
output['loss']

tensor(10.3750, dtype=torch.float16, grad_fn=<NllLossBackward0>)

In [ ]:
output['logits']

tensor([[-6.9297,  4.3750, -3.0938, -0.3516, -0.5249],
        [-6.0391,  3.3613, -2.1035, -1.0830, -1.1172]], dtype=torch.float16,
       grad_fn=<IndexBackward0>)

#### Métrica

Vamos a crear una función para obtener la métrica, que en este cáso va a ser el accuracy

In [ ]:
def predicted_labels(logits):
    percent = torch.softmax(logits, dim=1)
    predictions = torch.argmax(percent, dim=1)
    return predictions

def compute_accuracy(logits, labels):
    predictions = predicted_labels(logits)
    correct = (predictions == labels).float()
    return correct.mean()

Vamos a ver si lo calcula bien

In [ ]:
compute_accuracy(output['logits'], labels).item()

0.0

#### Optimizador

Como vamos a necesitar un optimizador, creamos uno

In [ ]:
from transformers import AdamW
LR = 2e-5
optimizer = AdamW(model.parameters(), lr=LR)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### Entrenamiento

Creamos el bucle de entrenamiento

In [ ]:
from tqdm import tqdm
EPOCHS = 1
accuracy = 0
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    progresbar = tqdm(train_loader, total=len(train_loader), desc=f'Epoch {epoch + 1}')
    for input_ids, at_mask, labels in progresbar:
        input_ids = input_ids.to(device)
        at_mask = at_mask.to(device)
        label = labels.to(device)
        output = model(input_ids=input_ids, attention_mask=at_mask, labels=label)
        loss = output['loss']
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progresbar.set_postfix({'train_loss': loss.item()})
    train_loss /= len(train_loader)
    progresbar.set_postfix({'train_loss': train_loss})
    model.eval()
    valid_loss = 0
    progresbar = tqdm(validation_loader, total=len(validation_loader), desc=f'Epoch {epoch + 1}')
    for input_ids, at_mask, labels in progresbar:
        input_ids = input_ids.to(device)
        at_mask = at_mask.to(device)
        labels = labels.to(device)
        output = model(input_ids=input_ids, attention_mask=at_mask, labels=labels)
        loss = output['loss']
        valid_loss += loss.item()
        step_accuracy = compute_accuracy(output['logits'], labels)
        accuracy += step_accuracy
        progresbar.set_postfix({'valid_loss': loss.item(), 'accuracy': step_accuracy.item()})
    valid_loss /= len(validation_loader)
    accuracy /= len(validation_loader)
    progresbar.set_postfix({'valid_loss': valid_loss, 'accuracy': accuracy})


Epoch 1: 100%|██████████| 3/3 [14:36<00:00, 292.15s/it, valid_loss=0, accuracy=1]


#### Uso del modelo

Vamos a probar el modelo que hemos entrenado

Primero tokenizamos un texto

In [ ]:
input_tokens = tokenize_function({"text": "I love this product. It is amazing."})
input_tokens['input_ids'].shape, input_tokens['attention_mask'].shape

(torch.Size([1, 768]), torch.Size([1, 768]))

Ahora se lo pasamos al modelo

In [ ]:
output = model(input_ids=input_tokens['input_ids'].to(device), attention_mask=input_tokens['attention_mask'].to(device))
output['logits']

tensor([[12.2422, -2.7715,  0.2188, -0.6567, -7.4570]], dtype=torch.float16,
       grad_fn=<IndexBackward0>)

Vemos las predicciones de esos logits

In [ ]:
predicted = predicted_labels(output['logits'])
predicted

tensor([0])